# 1. 直接使用Bert+mean pooling+ l2 距离 进行相似度计算


In [1]:
from transformers import AutoTokenizer,TFAutoModel,AutoConfig
import tensorflow as tf

In [2]:
# config = AutoConfig.from_pretrained('bert-base-chinese')
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
model = TFAutoModel.from_pretrained('bert-base-chinese')

2022-03-16 16:27:40.191371: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-16 16:27:47.058212: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5496 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2022-03-16 16:27:47.058937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5513 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['mlm___cls', 'nsp___c

In [3]:
sentence1 = "今天下午可能会下雨"
sentence2= '今天天气很晴朗'
sentence3= '天气预报说下午有雨'
sentence4= '北京是中国的首都'

sentence_list = [sentence1,sentence2,sentence3,sentence4]
encoded_inputs = tokenizer(sentence_list,return_tensors='tf',padding=True)
encoded_inputs

{'input_ids': <tf.Tensor: shape=(4, 11), dtype=int32, numpy=
array([[ 101,  791, 1921,  678, 1286, 1377, 5543,  833,  678, 7433,  102],
       [ 101,  791, 1921, 1921, 3698, 2523, 3252, 3306,  102,    0,    0],
       [ 101, 1921, 3698, 7564, 2845, 6432,  678, 1286, 3300, 7433,  102],
       [ 101, 1266,  776, 3221,  704, 1744, 4638, 7674, 6963,  102,    0]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4, 11), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4, 11), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]], dtype=int32)>}

In [5]:
sentence_embeddins = tf.math.reduce_mean(model(encoded_inputs).last_hidden_state,axis=1)

for i,s1 in enumerate(sentence_list):
    for j,s2 in enumerate(sentence_list):
        print(f'[{s1}]和[{s2}]的距离为：',tf.math.reduce_sum(tf.math.square(sentence_embeddins[i]-sentence_embeddins[j])).numpy())
 

[今天下午可能会下雨]和[今天下午可能会下雨]的距离为： 0.0
[今天下午可能会下雨]和[今天天气很晴朗]的距离为： 97.238594
[今天下午可能会下雨]和[天气预报说下午有雨]的距离为： 54.66022
[今天下午可能会下雨]和[北京是中国的首都]的距离为： 185.97879
[今天天气很晴朗]和[今天下午可能会下雨]的距离为： 97.238594
[今天天气很晴朗]和[今天天气很晴朗]的距离为： 0.0
[今天天气很晴朗]和[天气预报说下午有雨]的距离为： 91.689384
[今天天气很晴朗]和[北京是中国的首都]的距离为： 199.58101
[天气预报说下午有雨]和[今天下午可能会下雨]的距离为： 54.66022
[天气预报说下午有雨]和[今天天气很晴朗]的距离为： 91.689384
[天气预报说下午有雨]和[天气预报说下午有雨]的距离为： 0.0
[天气预报说下午有雨]和[北京是中国的首都]的距离为： 184.15388
[北京是中国的首都]和[今天下午可能会下雨]的距离为： 185.97879
[北京是中国的首都]和[今天天气很晴朗]的距离为： 199.58101
[北京是中国的首都]和[天气预报说下午有雨]的距离为： 184.15388
[北京是中国的首都]和[北京是中国的首都]的距离为： 0.0


# 2. 使用sentencebert+mean pooling + cosine距离 进行相似度计算


In [7]:
#https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

In [8]:
# Sentences we want sentence embeddings for
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]


#Encode query and docs
query_emb = encode(sentence_list)
doc_emb = encode(sentence_list)

#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1)).cpu()

for i,s1 in enumerate(sentence_list):
    for j,s2 in enumerate(sentence_list):
        print(f'[{s1}]和[{s2}]的相似度为：',scores[i,j].numpy())
 

[今天下午可能会下雨]和[今天下午可能会下雨]的相似度为： 1.0
[今天下午可能会下雨]和[今天天气很晴朗]的相似度为： 0.5500054
[今天下午可能会下雨]和[天气预报说下午有雨]的相似度为： 0.79448617
[今天下午可能会下雨]和[北京是中国的首都]的相似度为： 0.30826226
[今天天气很晴朗]和[今天下午可能会下雨]的相似度为： 0.5500054
[今天天气很晴朗]和[今天天气很晴朗]的相似度为： 0.9999999
[今天天气很晴朗]和[天气预报说下午有雨]的相似度为： 0.7026714
[今天天气很晴朗]和[北京是中国的首都]的相似度为： 0.21765988
[天气预报说下午有雨]和[今天下午可能会下雨]的相似度为： 0.79448617
[天气预报说下午有雨]和[今天天气很晴朗]的相似度为： 0.7026714
[天气预报说下午有雨]和[天气预报说下午有雨]的相似度为： 1.0000002
[天气预报说下午有雨]和[北京是中国的首都]的相似度为： 0.27188224
[北京是中国的首都]和[今天下午可能会下雨]的相似度为： 0.30826226
[北京是中国的首都]和[今天天气很晴朗]的相似度为： 0.21765988
[北京是中国的首都]和[天气预报说下午有雨]的相似度为： 0.27188224
[北京是中国的首都]和[北京是中国的首都]的相似度为： 1.0000002


# 分析阿里的数据集

In [9]:
import json
import tqdm
import numpy as np

In [10]:
s1_pairs=[]
s2_pairs=[]
labels=[]
with open('train.json') as f:
    for l in f.readlines():
        data = json.loads(l)
        s1_pairs.append(data['sentence1'])
        s2_pairs.append(data['sentence2'])
        labels.append(True if data['label']=='1' else False)

In [11]:
# 阿里数据集基本信息
print('训练数据集大小：',len(labels))
print(f'正例数：{sum(labels)},占比 {sum(labels)/len(labels)}')
print(f'负例数：{sum([not l for l in labels])},占比 {sum([not l for l in labels])/len(labels)}')

训练数据集大小： 34334
正例数：10573,占比 0.3079454767868585
负例数：23761,占比 0.6920545232131415


In [12]:
dev_s1_pairs=[]
dev_s2_pairs=[]
dev_labels=[]
with open('dev.json') as f:
    for l in f.readlines():
        data = json.loads(l)
        dev_s1_pairs.append(data['sentence1'])
        dev_s2_pairs.append(data['sentence2'])
        dev_labels.append(True if data['label']=='1' else False)

In [13]:
# 阿里数据集基本信息
print('测试数据集大小：',len(dev_labels))
print(f'正例数：{sum(dev_labels)},占比 {sum(dev_labels)/len(dev_labels)}')
print(f'负例数：{sum([not l for l in dev_labels])},占比 {sum([not l for l in dev_labels])/len(dev_labels)}')

测试数据集大小： 4316
正例数：1338,占比 0.31000926784059313
负例数：2978,占比 0.6899907321594069


# 模型效果评估-测试集

In [14]:
dev_s1_embs = encode(dev_s1_pairs)
dev_s2_embs = encode(dev_s2_pairs)

In [15]:
scores = torch.diag(torch.mm(dev_s1_embs,dev_s2_embs.transpose(0,1)))

In [16]:
correct = 0
tp=0
tn=0
fp=0
fn=0
for score,label in tqdm.tqdm(zip(scores,dev_labels)):
    pred = True if  score>0.5 else False
    if label == pred:
        correct+=1
        if label :
            tp+=1
        else:
            tn+=1
    else:
        if pred :
            fp+=1
        else:
            fn+=1

4316it [00:00, 144188.80it/s]


In [18]:
print('测试集正确率',correct/len(dev_labels))
print(f'TP:{tp}, TN:{tn}, FP:{fp}, FN:{fn}')

测试集正确率 0.3561167747914736
TP:1256, TN:281, FP:2697, FN:82
